In [226]:
import mysql.connector as mysql
from config import host, user, password
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
import math

In [2]:
con = mysql.connect(user=user,password=password,host=host,database='team3')

In [3]:
cursor = con.cursor()

## 1. Liczba spotkań każdego miesiąca

In [4]:
query1 = "SELECT * FROM opponents"

In [5]:
cursor.execute(query1)
results = cursor.fetchall()
matches = pd.DataFrame(results,columns=['team_name','date','opponent_name'])

In [6]:
indexes = []
for index, row in matches.iterrows():
    if index == len(matches) -1:
        break
    if row['team_name'] == matches.loc[index+1,'opponent_name']:
        indexes.append(index)

In [7]:
matches.drop(indexes,inplace=True)

In [8]:
matches['month'] = pd.DatetimeIndex(matches['date']).month
matches['year'] = pd.DatetimeIndex(matches['date']).year

In [9]:
group_matches = matches.groupby(by=['year','month']).count()['team_name']

In [10]:
match_freq = pd.DataFrame(list(group_matches.index), columns=['year','month'])
match_freq['day'] = 1
match_freq['date'] = pd.to_datetime(match_freq)
match_freq = match_freq[['date']]
match_freq['freq'] = group_matches.values

In [116]:
date_range = list(pd.date_range('2019-06-01','2023-05-30',freq='MS'))

In [117]:
for date in date_range:
    if date not in list(match_freq['date']):
        row = {'date':date,'freq':0}
        match_freq = match_freq.append(row,ignore_index=True)

In [118]:
match_freq.sort_values(by=['date'],inplace=True)
match_freq.reset_index(inplace=True)

In [239]:
fig = px.line(match_freq, x="date", y="freq", title='Liczba spotkań każdego miesiąca')
fig.update_layout(xaxis_title='czas',
                  yaxis_title='liczba meczy',
                  title_x=0.5)
fig.show()

Na powyższym wykresie widać, że przez pierwsze pół roku działalności klubu nasze drużyny nie rozgrywały żadnych meczy. Pierwsze spotkania odbyły się dopiero na  początku roku 2020. Niestety rozwój naszego klubu zatrzymał lockdown spowodowany pandemią covid 19 i od początku marca do końca czerwca roku 2020 również nie rozgrywaliśmy meczy. 

Od lipca 2020 roku obserwujemy już powolny rozwój klubu. Liczba spotkań przejawia tendencję rosnącą, przy czym najwięcej meczy (ponad 50) rozegranych zostało niedawno, bo w maju 2022 roku.

Na wykresie przedstawiona została także liczba meczy zaplanowanych na każdy miesiąc przyszłego sezonu.

## 2. Bilans wygranych i przegranych meczy

In [15]:
query2 = '''SELECT team_name, team_score, opponent_score, team_score > opponent_score, team_score < opponent_score 
            FROM matches WHERE team_score IS NOT NULL'''

In [16]:
cursor.execute(query2)
results = cursor.fetchall()
scores = pd.DataFrame(results,columns=['team_name','team_score','opponent_score','match_won','match_lost'])

In [17]:
scores = scores.groupby(by=['team_name']).sum()[['match_won','match_lost']]
scores.sort_values(by=['team_name'],inplace=True)

In [74]:
lost = scores['match_lost']
won = scores['match_won']
teams = list(scores.index)

In [240]:

layout = go.Layout(yaxis=go.layout.YAxis(
                        ticktext=teams,
                        range=[0, len(teams)],
                        title='Teams'),
                   xaxis=go.layout.XAxis(
                        range=[-lost.max()-1, won.max()+1],
                        tickvals=[i for i in range(-30,40,10)],
                        ticktext=[str(abs(i)) for i in range(-30,40,10)],
                        title='Number'),
                   barmode='overlay',
                   autosize=False,
                   width=800,
                   height=1100,
                   title='Bilans wygranych i przegranych meczy',
                   title_x=0.5,
                   bargap=0.1)

data = [go.Bar(y=teams,
               x=won,
               orientation='h',
               name='wygrane',
               text=won,
               hoverinfo='x+y',
               marker=dict(color='darkgreen'),
               textangle=0, 
               #textposition="outside"
               ),
        go.Bar(y=teams,
               x=-lost,
               orientation='h',
               name='przegrane',
               text=lost,
               hoverinfo='text+y',
               marker=dict(color='darkred'),
               textangle=0, 
               #textposition="outside"
               )]
#fig.update_traces(textangle=0, textposition="outside")

pyo.iplot(dict(data=data, layout=layout))

Powyższy wykres piramidowy przedstawia liczbę wygranych i przegranych meczy dla każdej z naszych drużyn. Widać duże rozbieżności pomiędzy drużynami. 

Sześć drużyn ('Baby Broomers', 'Curlers Terriers', 'Calgary Metros', 'Montreal Vipers', 'Rolling Stons' i 'Super Atlantics') rozegrało nie więcej niż sześć meczy, co może wynikać z braku możliwości skopletowania składu. Natomiast osiem drużyn odbyło ponad 30 spotkań.

Istenieją też znaczne różnice pomiędzy bilansem wygranych i przegranych meczy. Do najlepszych drużyn zdecydowanie możemy zaliczyć 'Vancouver Fury', którzy wygrali 82% z rozegranych meczy oraz 'Montreal Bruins' którzy mają najwięcej wygranych meczy ze wszystkich drużyn i niewielką ilość przegranych. Wysoko plasują się także 'Calgary Rockets', którzy na 20 rozgranych meczy nie przegrali ani jednego. Do innych drużyn, które wygrały wszystkie swoje mecze możemy także zaliczyć 'Toronto Olympics', 'Montreal Braves', 'Curlers Terriers' oraz 'Calgary Metros'. Musimy jednak wziąć pod uwagę, że dwie ostatnie drużyny rozegrały mniej meczy w porównaniu do innych drużyn o idealnej skuteczności.

Najgorszymi drużynami okazały się 'Vancouver Islanders' - przegrali wszytskie 32 mecze! Dużą przewagę przegranych do wygranych meczy mają także 'Toronto Eagels', 'Stone Mariners' oraz 'Ice Sabres'.

## 3. Średni wiek graczy

In [228]:
query3 = "SELECT birthdate, join_date, retire_date, age_category FROM personal_info INNER JOIN teams USING(team_name)"

In [229]:
cursor.execute(query3)
results = cursor.fetchall()
dates = pd.DataFrame(results,columns=['birthdate','join_date','retire_date','age_category'])

In [230]:
junior_age = pd.DataFrame(columns=['date','mean_age'])
adult_age = pd.DataFrame(columns=['date','mean_age'])

In [231]:
date_range2 = list(pd.date_range('2019-06-01','2022-05-30',freq='W'))

In [232]:
for date in date_range2:
    j_ages = []
    a_ages = []
    for index, row in dates.iterrows():
        age = math.floor((date -  pd.to_datetime(row['birthdate']))/np.timedelta64(1, 'Y'))
        if (row['join_date'] <= date) and ((row['retire_date'] == None) or (row['retire_date'] >= date)):
            if row['age_category'] == 'junior':
                j_ages.append(age)
            elif row['age_category'] == 'adult':
                a_ages.append(age)

    if j_ages != []:
        row = {'date':date,'mean_age':np.mean(j_ages)}
        junior_age = junior_age.append(row,ignore_index=True)

    if a_ages != []:
        row = {'date':date,'mean_age':np.mean(a_ages)}
        adult_age = adult_age.append(row,ignore_index=True)

In [241]:
fig = px.line(junior_age, x="date", y="mean_age", title='Średni wiek juniorów')
fig.update_layout(xaxis_title='czas',
                  yaxis_title='średin wiek',
                  title_x=0.5)
fig.show()

In [242]:
fig = px.line(adult_age, x="date", y="mean_age", title='Średni wiek dorosłych graczy')
fig.update_layout(xaxis_title='czas',
                  yaxis_title='średni wiek',
                  title_x=0.5)
fig.show()

Dwa powyższe wykresy przedstawiają średni wiek w zależności od czasu dla graczy z kategorii juniorów i dorosłych. 

W obu przypadkach zauważalna jest ogólna tendencja wzrostowa, co jest logiczne biorąc pod uwagę, że nasi gracze z roku na rok są coraz starsi. Widać jednak, że miejscami średni wiek znacząco spada, zwłaszcza na początku naszej działalności, co jest spowodowane odchodzeniem graczy z klubu. Spadki są bardzo widoczne również ze względu na niewielką ilość graczy na początku istnienia klubu.

Dla dorosłych obserwujemy jednak dwa większe spadki tekże w późniejszych miesiącach - w połowie roko 2020 i na początku 2022.

Możemy również zauważyć, że średni wiek juniorów nie przekracza 14 lat, a pierwsi gracze, który dołączyli mieli śrdednio 9 lat. 
Dla dorosłych natomiast średni wiek nie przekracza 40 lat, a pierwsi gracze mieli średnio 28 lat.

## 4. Analiza ilości graczy w zależności od czasu

In [209]:
query4 = '''SELECT gender, join_date, retire_date 
            FROM personal_info 
            INNER JOIN people USING(person_id) 
            INNER JOIN gender USING(first_name) 
            WHERE team_name IS NOT NULL'''

In [214]:
cursor.execute(query4)
results = cursor.fetchall()
players = pd.DataFrame(results,columns=['gender','join_date','retire_date'])

In [263]:
women_number = pd.DataFrame(columns=['date','number','gender'])
men_number = pd.DataFrame(columns=['date','number','gender'])

In [264]:
for date in date_range2:
    
    women_counter = 0
    men_counter = 0

    for index, row in players.iterrows():
        if (row['join_date'] <= date) and ((row['retire_date'] == None) or (row['retire_date'] >= date)):
            if row['gender'] == 'F':
                women_counter += 1
            elif row['gender'] == 'M':
                men_counter += 1

    row_w = {'date':date,'number':women_counter,'gender':'female'}
    row_m = {'date':date,'number':men_counter,'gender':'male'}
    women_number = women_number.append(row_w,ignore_index=True)
    men_number = men_number.append(row_m,ignore_index=True)


In [265]:
fig = px.line(pd.concat([women_number,men_number]), x="date", y="number", color='gender',title='Ilość graczy w zależności od czasu')
fig.update_layout(xaxis_title='czas',
                  yaxis_title='ilość graczy',
                  title_x=0.5)
fig.show()

Na powyższym wykresie obserwujemy tendencję wzrostową ilości graczy w klubie, zarówno dla kobiet i dla mężczyzn, przyczym liczba kobiet nigdy nie przekracza ilości mężczyzn.